# Практическая работа

# Задача

Один из способов повысить эффективность взаимодействия банка с клиентами — отправлять предложение о новой услуге не всем клиентам, а только некоторым, которые выбираются по принципу наибольшей склонности к отклику на это предложение.

Задача заключается в том, чтобы предложить алгоритм, который будет выдавать склонность клиента к положительному или отрицательному отклику на предложение банка. Предполагается, что, получив такие оценки для некоторого множества клиентов, банк обратится с предложением только к тем, от кого ожидается положительный отклик.


Для решения этой задачи загрузите файлы из базы в Postgres.
Эта БД хранит информацию о клиентах банка и их персональные данные, такие как пол, количество детей и другие.

Описание таблиц с данными представлено ниже.


**D_work**

Описание статусов относительно работы:
- ID — идентификатор социального статуса клиента относительно работы;
- COMMENT — расшифровка статуса.


**D_pens**

Описание статусов относительно пенсии:
- ID — идентификатор социального статуса;
- COMMENT — расшифровка статуса.


**D_clients**

Описание данных клиентов:
- ID — идентификатор записи;
- AGE	— возраст клиента;
- GENDER — пол клиента (1 — мужчина, 0 — женщина);
- EDUCATION — образование;
- MARITAL_STATUS — семейное положение;
- CHILD_TOTAL	— количество детей клиента;
- DEPENDANTS — количество иждивенцев клиента;
- SOCSTATUS_WORK_FL	— социальный статус клиента относительно работы (1 — работает, 0 — не работает);
- SOCSTATUS_PENS_FL	— социальный статус клиента относительно пенсии (1 — пенсионер, 0 — не пенсионер);
- REG_ADDRESS_PROVINCE — область регистрации клиента;
- FACT_ADDRESS_PROVINCE — область фактического пребывания клиента;
- POSTAL_ADDRESS_PROVINCE — почтовый адрес области;
- FL_PRESENCE_FL — наличие в собственности квартиры (1 — есть, 0 — нет);
- OWN_AUTO — количество автомобилей в собственности.


**D_agreement**

Таблица с зафиксированными откликами клиентов на предложения банка:
- AGREEMENT_RK — уникальный идентификатор объекта в выборке;
- ID_CLIENT — идентификатор клиента;
- TARGET — целевая переменная: отклик на маркетинговую кампанию (1 — отклик был зарегистрирован, 0 — отклика не было).
    
    
**D_job**

Описание информации о работе клиентов:
- GEN_INDUSTRY — отрасль работы клиента;
- GEN_TITLE — должность;
- JOB_DIR — направление деятельности внутри компании;
- WORK_TIME — время работы на текущем месте (в месяцах);
- ID_CLIENT — идентификатор клиента.


**D_salary**

Описание информации о заработной плате клиентов:
- ID_CLIENT — идентификатор клиента;
- FAMILY_INCOME — семейный доход (несколько категорий);
- PERSONAL_INCOME — личный доход клиента (в рублях).


**D_last_credit**

Информация о последнем займе клиента:
- ID_CLIENT — идентификатор клиента;
- CREDIT — сумма последнего кредита клиента (в рублях);
- TERM — срок кредита;
- FST_PAYMENT — первоначальный взнос (в рублях).


**D_loan**

Информация о кредитной истории клиента:
- ID_CLIENT — идентификатор клиента;
- ID_LOAN — идентификатор кредита.

**D_close_loan**

Информация о статусах кредита (ссуд):
- ID_LOAN — идентификатор кредита;
- CLOSED_FL — текущий статус кредита (1 — закрыт, 0 — не закрыт).

Ниже представлен минимальный список колонок, которые должны находиться в итоговом датасете после склейки и агрегации данных. По своему усмотрению вы можете добавить дополнительные к этим колонки.

    - AGREEMENT_RK — уникальный идентификатор объекта в выборке;
    - TARGET — целевая переменная: отклик на маркетинговую кампанию (1 — отклик был зарегистрирован, 0 — отклика не было);
    - AGE — возраст клиента;
    - SOCSTATUS_WORK_FL — социальный статус клиента относительно работы (1 — работает, 0 — не работает);
    - SOCSTATUS_PENS_FL — социальный статус клиента относительно пенсии (1 — пенсионер, 0 — не пенсионер);
    - GENDER — пол клиента (1 — мужчина, 0 — женщина);
    - CHILD_TOTAL — количество детей клиента;
    - DEPENDANTS — количество иждивенцев клиента;
    - PERSONAL_INCOME — личный доход клиента (в рублях);
    - LOAN_NUM_TOTAL — количество ссуд клиента;
    - LOAN_NUM_CLOSED — количество погашенных ссуд клиента.


Будьте внимательны при сборке датасета: это реальные банковские данные, в которых могут наблюдаться дубли, некорректно заполненные значения или значения, противоречащие друг другу. Для получения качественной модели необходимо предварительно очистить датасет от такой информации.

## Задание 1

В предыдущем задании вы собрали всю информацию о клиентах в одну таблицу, где одна строчка соответствует полной информации об одном клиенте.

Загрузите эту таблицу.

In [29]:
import streamlit as st
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [38]:
df = pd.read_csv('df_prepare.csv')

In [39]:
numerical = ['AGE',
             'WORK_TIME',
             'PERSONAL_INCOME',
             'CREDIT',
             'TERM',
             'FST_PAYMENT']

id_list = ['AGREEMENT_RK',
           'ID_CLIENT',
           'TARGET',
           'ID_LOAN']

cat = list(df.columns)
cat = list(set(cat) - set(numerical) - set(id_list))

In [40]:
#Закодируем категориальные фичи
df = pd.get_dummies(df, columns=cat)

In [41]:
df

,AGREEMENT_RK,ID_CLIENT,TARGET,AGE,WORK_TIME,PERSONAL_INCOME,CREDIT,TERM,FST_PAYMENT,ID_LOAN,...,JOB_DIR_Кадровая служба и секретариат,JOB_DIR_Пр-техн. обесп. и телеком.,JOB_DIR_Реклама и маркетинг,JOB_DIR_Служба безопасности,JOB_DIR_Снабжение и сбыт,JOB_DIR_Участие в основ. деятельности,JOB_DIR_Юридическая служба,OWN_AUTO_0,OWN_AUTO_1,OWN_AUTO_2
0,0.000000,0.000000,0,0.608696,0.395226,0.124475,0.122100,0.157895,0.461703,0.000000,...,0,0,0,0,0,0,0,1,0,0
1,0.000005,0.000066,0,0.239130,0.625540,0.299580,0.399878,0.157895,0.213504,0.000047,...,0,0,0,0,0,1,0,1,0,0
2,0.000024,0.000131,0,0.673913,0.605864,0.224535,0.633415,0.473684,0.213504,0.000095,...,0,0,0,0,0,1,0,1,0,0
3,0.000024,0.000131,0,0.673913,0.605864,0.224535,0.633415,0.473684,0.213504,0.000142,...,0,0,0,0,0,1,0,1,0,0
4,0.000042,0.000197,0,0.391304,0.700644,0.624775,0.132109,0.157895,0.266880,0.000189,...,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19354,0.999947,0.999737,0,0.086957,0.529343,0.299580,0.233008,0.368421,0.080064,0.999763,...,0,0,0,0,0,1,0,1,0,0
19355,0.999947,0.999737,0,0.086957,0.529343,0.299580,0.233008,0.368421,0.080064,0.999811,...,0,0,0,0,0,1,0,1,0,0
19356,0.999977,0.999803,0,0.652174,0.690511,0.249550,0.142267,0.157895,0.213504,0.999858,...,0,0,0,0,0,1,0,1,0,0
19357,0.999979,0.999869,0,0.021739,0.529343,0.149490,0.014245,0.157895,0.042701,0.999905,...,1,0,0,0,0,0,0,1,0,0


Разбейте данные на тренировочную и тестовую часть в пропорции 80% к 20%, зафиксируйте `random_state = 42`.

In [45]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['TARGET'], axis=1), df['TARGET'], test_size=0.2, random_state=42)

На тренировочных данных обучите линейную модель классификации для предсказания целевой переменной (столбец `TARGET`).

Сделайте прогноз вероятности отклика на рекламную кампанию для тестовых данных.

In [ ]:
# your code here

Переведите вероятности в классы по стандартному порогу (0.5) и на тестовом наборе данных вычислите метрики:

* accuracy
* precision
* recall
* f1-score

In [ ]:
# your code here

Целевая метрика для задачи - полнота, так как нам нужно найти максимум клиентов, кто может откликнуться на рекламу.

Но при этом точность не должна просесть, поэтому за ней тоже следим.

Разбейте тренировочные данные на `train` и `val` части в пропорции 3 к 1.

В цикле:

* переберите пороги от 0 до 1 с шагом 0.01
* вычислите для каждого порога значение метрик precision и recall
* подберите такой порог, при котором recall не меньше 0.66, а точность максимальна.

In [ ]:
# your code here

Для выбранного порога посчитайте все метрики на тестовых данных. Сильно ли они отличаются от метрик на валидации?

In [ ]:
# your code here

Выведите на экран в виде таблицы топ-6 признаков с наибольшими по модулю весами модели.

In [ ]:
# your code here

## Задание 2

Добавьте в Streamlit-приложение визуализацию результатов модели:

* опцию выбора порога и вывод метрик качества в зависимости от выбранного порога

* вывод прогноза модели на выбранном объекте (клиенте) - вероятность отклика на рекламу.

## Бонус

Попробуйте применить другие модели классификации для решения этой задачи (любые какие знаете).

Удалось ли добиться улучшения качества модели?

In [ ]:
# your code here